# Scrapping with BeautifulSoup 

Scrapper des données sur le Web à l'aide des librairies **BeautifulSoup** et **Requests**

In [9]:
import requests
from bs4 import BeautifulSoup

## Requests, récuprer le code d'un site web

**requests** est un module python qui sert à faire des requêtes sur le web via les protocoles HTTP  

Avec se module nous allons pouvoir récuperer le code HTML d'un site web via son URL

Quelques méthodes de *`requests`*:

>* *`.get('url')`* LA commande qui permet d'aller chercher via une URL le format brut HTML sous forme d'un objet "HTTPResponse" du module urllib3.  Possible argument *`auth=('username','password')`* si il y a besoins d'une authentification  
>* *`.post`* permet d'envoyer des données vers le siteweb comme un *username* et un *psswd*  à travers un *`dict`*, ou même des img..
>* *`sess = requests.Session()`*; puis  Pour effectuer plusieurs requêtes sur un site, qui permet de conserver les identification et les cookies pendant l'exécution de toutes les requêtes. L'objet *`Session`* ici *`sess`* possède toute les méthodes de `requests` dont *`.get`* par exemple.

On peut lire la réponse de *`.get`* avec:
>* *`r.text`* Retourne le contenu en unicode
>* *`r.content`* Retourne le contenu en bytes
>* *`r.json`* Retourne le contenu sous forme json
>* *`r.headers`* Retourne le headers sous forme de dictionnaire 

In [15]:
html = requests.get('https://en.wikipedia.org/wiki/Main_Page') #auth=('username','password') si besoin

In [16]:
#html.headers

## BeautifulSoup pour analyser le HTML  
Començons par créer un Objet BeautfulSoup avec la réponse récupérée par *`.get()`* avec la méthode *`.content`* de *`requests`*, pour pouvoir ensuite utiliser toutes les méthodes d'analyse de HTML de BS.  
On lui passe en arg *`html.parser`* pour lui signifier que c'est bien du html.

In [14]:
soup = BeautifulSoup(html.content, "html.parser") 

Une fois le code HTML dans un objet BeatifuSoup on va pouvoir commencer à l'analyser avec différentes méthodes..  
Un petit rappel sur ce que l'on peut identifier dans du HTML:

- balises, attribut, valeur et le contenu .... entres autres

voire le code HTML avec *`.prettify`*

In [264]:
#soup.prettify

### Naviguer avec les balises (tags)

<img src="img.png"/>

Quelques méthodes de BeautifulSoup pour accéder au contenu d'une page web:
>- par Balises (tag) *`soup`*: *`.p`, `.a`, `.h1`,`.div`*, *`li`* ect... ou avec *`.find('a')`*
>- puis Attribut id *`.tag.id`* ou *`.find('a', id='content')`*
>- Attribut class *`.tag.class_`* ou *`find('a', class_='client')`*
>- Attribut name *`.tag.attrs`* ou *`find('a', attrs{'name':'content'})`*  

>- *`findAll` ou `find_all`* pour trouver les occurences d'un element, s'utilise comme *`.find`*

In [236]:
soup.p.a

<a href="/wiki/Abbasid_invasion_of_Asia_Minor_(806)" title="Abbasid invasion of Asia Minor (806)">Abbasid invasion of Asia Minor</a>

In [63]:
soup.div.attrs

{'id': 'mw-page-base', 'class': ['noprint']}

In [112]:
soup.find('a')

<a id="top"></a>

- Trouver les les liens désignés par *`href`* dans la balise *`<a`*

In [237]:
#le premier lien
soup.p.a['href']

'/wiki/Abbasid_invasion_of_Asia_Minor_(806)'

pour récupérer tous les liens de la page; contenus dans des balises *`<a`* notés avec l'attribut *`href`*

In [289]:
liens = [i.get('href') for i in soup.find_all('a')[3:150]]

In [290]:
liens

['/wiki/Wikipedia',
 '/wiki/Free_content',
 '/wiki/Encyclopedia',
 '/wiki/Wikipedia:Introduction',
 '/wiki/Special:Statistics',
 '/wiki/English_language',
 '/wiki/Portal:Arts',
 '/wiki/Portal:Biography',
 '/wiki/Portal:Geography',
 '/wiki/Portal:History',
 '/wiki/Portal:Mathematics',
 '/wiki/Portal:Science',
 '/wiki/Portal:Society',
 '/wiki/Portal:Technology',
 '/wiki/Wikipedia:Contents/Portals',
 '/wiki/File:Heraqla,SE.jpg',
 '/wiki/Abbasid_invasion_of_Asia_Minor_(806)',
 '/wiki/Abbasid_Caliphate',
 '/wiki/Byzantine_Empire',
 '/wiki/Anatolia',
 '/wiki/List_of_Byzantine_emperors',
 '/wiki/Nikephoros_I',
 '/wiki/Tribute',
 '/wiki/Harun_al-Rashid',
 '/wiki/List_of_Abbasid_caliphs',
 '/wiki/Heraclea_Cybistra',
 '/wiki/Jizya',
 '/wiki/Staurakios',
 '/wiki/Fourth_Fitna',
 '/wiki/Battle_of_Pliska',
 '/wiki/Abbasid_invasion_of_Asia_Minor_(806)',
 '/wiki/Vision_in_White',
 '/wiki/Horologium_(constellation)',
 '/wiki/Razing_of_Friesoythe',
 '/wiki/Wikipedia:Today%27s_featured_article/April_2020

In [279]:
#on peut aussi faire
#l=[]
#for i in soup.findAll('a'):
#    l.append(a["href"])
#l

- afficher les parents du tag *`p`*

In [76]:
for p in soup.p.parents:
    print(p.name)

div
td
tr
tbody
table
div
div
div
div
body
html
[document]


- chercher les parents du tag *`.li`*

In [ ]:
#for p in soup.li.parents:
#    print(p)

### Accéder au texte 

>- *`.text`*
>- *`.string`*
>- *`.get_text()`* on peut lui passer un arg comme *`"|"`* qui séparer 

In [96]:
soup.p.a.string

'Abbasid invasion of Asia Minor'

In [106]:
soup.p.text[:151]

'The Abbasid invasion of Asia Minor in 806 CE was an attack by the Abbasid Caliphate against the Byzantine Empire in southeastern and central Asia Minor'

- *`soup.get_text()`* va afficher tous le texte contenue dans *`soup`*

In [132]:
#soup.get_text()

## CSS selector  
>- *`.select()`* Pour capturer les élement contenue dans un fichier .css

In [262]:
soup.select('a')[15]

<a href="/wiki/Portal:Society" title="Portal:Society">Society</a>

In [263]:
new_links = {}

#follow each link:
for link in liens:
    webpage = requests.get(link)
    new_soup = BeautifulSoup(webpage.content, "html.parser")
    
    new_l = new_soup.select(".name")[0]
    # dans le contenue de la pageweb on select du CSS(les tags de la classe ".name")
    #print(type(turtle_name))
    new_links[turtle_name] = []#add vlaue in an empty list


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

In [ ]:
#res = requests.get("https://en.wikipedia.org/wiki/List_of_naval_battles")
soup = bs(res.text, "html.parser")
naval_battles = {}
for link in soup.find_all("a"):
    url = link.get("href", "")
    if "/wiki/Battle" in url:
        naval_battles[link.text.strip()] = url

print(naval_battles)

In [292]:
soup.select(".header")

[]

## références
**Requests**
- http://fr.python-requests.org/en/latest/user/quickstart.html#requetes-post-avancees
- https://requests-fr.readthedocs.io/en/latest/user/advanced.html
- http://dridk.me/python-requests.html

**BeautifulSoup**
- http://jhroy.ca/uqam/edm5240/BeautifulSoup-DocAbregee.pdf
- http://sametmax.com/parser-du-html-avec-beautifulsoup/

- https://www.crummy.com/software/BeautifulSoup/bs4/doc/  (doc officiel)
